# Progetto Lar Generators (Studio esecutivo)

Per avviare una istanza Jupyter con 8 threads, eseguire il seguente codice dal proprio terminale e selezionare il kernel appena creato.

```julia
using IJulia
IJulia.installkernel("Julia 8 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "8",
))
````


In [ ]:
Threads.nthreads()

## Codice versione originale

In [ ]:
using LinearAlgebraicRepresentation, SparseArrays
Lar = LinearAlgebraicRepresentation;
using IntervalTrees, LinearAlgebra

function pointInPolygonClassification(V, EV)
    function pointInPolygonClassification0(pnt)
        x, y = pnt
        xmin, xmax, ymin, ymax = x, x, y, y
        tilecode = Lar.setTile([ymax, ymin, xmax, xmin])
        count, status = 0, 0

        for (k, edge) in enumerate(EV)
            p1, p2 = V[:, edge[1]], V[:, edge[2]]
            (x1, y1), (x2, y2) = p1, p2
            c1, c2 = tilecode(p1), tilecode(p2)
            c_edge, c_un, c_int = c1 ⊻ c2, c1 | c2, c1 & c2

            if (c_edge == 0) & (c_un == 0)
                return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge)
                return "p_on"
            elseif c_edge == 3
                if c_int == 0
                    return "p_on"
                elseif c_int == 4
                    count += 1
                end
            elseif c_edge == 15
                x_int = ((y - y2) * (x1 - x2) / (y1 - y2)) + x2
                if x_int > x
                    count += 1
                elseif x_int == x
                    return "p_on"
                end
            elseif (c_edge == 13) & ((c1 == 4) | (c2 == 4))
                crossingTest(1, 2, status, count)
            elseif (c_edge == 14) & ((c1 == 4) | (c2 == 4))
                crossingTest(2, 1, status, count)
            elseif c_edge == 7
                count += 1
            elseif c_edge == 11
                count = count
            elseif c_edge == 1
                if c_int == 0
                    return "p_on"
                elseif c_int == 4
                    crossingTest(1, 2, status, count)
                end
            elseif c_edge == 2
                if c_int == 0
                    return "p_on"
                elseif c_int == 4
                    crossingTest(2, 1, status, count)
                end
            elseif (c_edge == 4) & (c_un == c_edge)
                return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge)
                return "p_on"
            elseif c_edge == 5
                if (c1 == 0) | (c2 == 0)
                    return "p_on"
                else
                    crossingTest(1, 2, status, count)
                end
            elseif c_edge == 6
                if (c1 == 0) | (c2 == 0)
                    return "p_on"
                else
                    crossingTest(2, 1, status, count)
                end
            elseif (c_edge == 9) & ((c1 == 0) | (c2 == 0))
                return "p_on"
            elseif (c_edge == 10) & ((c1 == 0) | (c2 == 0))
                return "p_on"
            end
        end
        if (round(count) % 2) == 1
            return "p_in"
        else
            return "p_out"
        end
    end
    return pointInPolygonClassification0
end

function spaceindex(point3d::Array{Float64,1})::Function
    function spaceindex0(model::Lar.LAR)::Array{Int,1}
        V, CV = copy(model[1]), copy(model[2])
        V = [V point3d]
        dim, idx = size(V)
        push!(CV, [idx, idx, idx])
        cellpoints = [V[:, CV[k]]::Lar.Points for k = 1:length(CV)]

        bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
        xboxdict = Lar.coordintervals(1, bboxes)
        yboxdict = Lar.coordintervals(2, bboxes)

        xs = IntervalTrees.IntervalMap{Float64,Array}()
        for (key, boxset) in xboxdict
            xs[tuple(key...)] = boxset
        end
        ys = IntervalTrees.IntervalMap{Float64,Array}()
        for (key, boxset) in yboxdict
            ys[tuple(key...)] = boxset
        end
        xcovers = Lar.boxcovering(bboxes, 1, xs)
        ycovers = Lar.boxcovering(bboxes, 2, ys)
        covers = [intersect(pair...) for pair in zip(xcovers, ycovers)]

        pointcover = setdiff(covers[end], [idx + 1])
        return pointcover[1:end-1]
    end
    return spaceindex0
end

function rayintersection(point3d)
    function rayintersection0(V, FV, face::Int)
        l0, l = point3d, [0, 0, 1.0]
        ps = V[:, FV[face]]  # face points
        p0 = ps[:, 1]
        v1, v2 = ps[:, 2] - p0, ps[:, 3] - p0
        n = LinearAlgebra.normalize(cross(v1, v2))

        denom = dot(n, l)
        if (abs(denom) > 1e-8) #1e-6
            p0l0 = p0 - l0
            t = dot(p0l0, n) / denom
            if t > 0
                return l0 + t * l
            end
        else
            return false
        end
    end
    return rayintersection0
end


function planemap(V, copEV, copFE, face)
    fv, edges = Lar.vcycle(copEV, copFE, face)
    function planemap0(point)
        vs = V[:, fv]
        point = point .- vs[:, 1]
        vs = vs .- vs[:, 1]
        u, v = edges[1]
        z, w = [[z, w] for (z, w) in edges if z == v][1]
        v1 = vs[:, u] - vs[:, v]
        v2 = vs[:, w] - vs[:, v]
        v3 = cross(v2, v1)
        M = [v1 v2 v3]
        vs = inv(M) * [vs point]
        outvs = vs[1:2, 1:end-1]
        outpoint = vs[1:2, end]
        return outvs, edges, outpoint
    end
    return planemap0
end

function settestpoints(V, EV, FV, Fs, copEV, copFE)
    fdict = Dict(zip(Fs, 1:length(Fs)))
    countKey = 1
    f = Fs[1] #prima faccia numerata dell' i-esimo solido
    e = findnz(copFE[f, :])[1][1] # first (global) edge of first (global) face FE[f][1]
    f1 = findnz(copFE[:, e])[1][countKey]
    while !haskey(fdict, f1)
        countKey = countKey + 1
        f1 = findnz(copFE[:, e])[1][countKey]
    end
    f2 = findnz(copFE[:, e])[1][countKey+1]
    while !haskey(fdict, f2)
        countKey = countKey + 1
        f2 = findnz(copFE[:, e])[1][countKey]
    end
    v1, v2 = findnz(copEV[e, :])[1] # two (global) verts incident on it
    V1 = FV[fdict[f1]]
    V2 = FV[fdict[f2]]
    v1, v2 = intersect(V1, V2) # verified ... !
    t1 = V[:, v1], V[:, v2], V[:, [v for v in V1 if v ≠ v1 && v ≠ v2][1]]
    t2 = V[:, v2], V[:, v1], V[:, [v for v in V2 if v ≠ v1 && v ≠ v2][1]]
    n1 = LinearAlgebra.normalize(cross(t1[2] - t1[1], t1[3] - t1[1]))
    n2 = LinearAlgebra.normalize(cross(t2[2] - t2[1], t2[3] - t2[1]))
    p0 = (V[:, v1] + V[:, v2]) ./ 2
    n = n1 + n2
    ϵ = 1.0e-4
    ptest1 = p0 + ϵ * n
    ptest2 = p0 - ϵ * n
    return ptest1, ptest2
end


function testinternalpoint(V, EV, FV)
    copEV = Lar.lar2cop(EV)
    copFV = Lar.lar2cop(FV)
    copFE = copFV * copEV'
    I, J, Val = findnz(copFE)
    triple = zip([(i, j, 1) for (i, j, v) in zip(I, J, Val) if v == 2]...)
    I, J, Val = map(collect, triple)
    Val = convert(Array{Int8,1}, Val)
    copFE = sparse(I, J, Val)
    function testinternalpoint0(testpoint)
        intersectedfaces = Int64[]

        faces = spaceindex(testpoint)((V, FV))
        depot = []

        for face in faces
            value = rayintersection(testpoint)(V, FV, face)
            if typeof(value) == Array{Float64,1}
                push!(depot, (face, value))
            end
        end

        for (face, point3d) in depot
            vs, edges, point2d = planemap(V, copEV, copFE, face)(point3d)
            classify = pointInPolygonClassification(vs, edges)
            inOut = classify(point2d)
            if inOut != "p_out"
                push!(intersectedfaces, face)
            end
        end
        return intersectedfaces
    end
    return testinternalpoint0
end


function getinternalpoint(V, EV, FV, Fs, copEV, copFE)

    ptest1, ptest2 = settestpoints(V, EV, FV, Fs, copEV, copFE)
    intersectedfaces = Int64[]

    dep1, dep2 = [], []

    for (f, face) in enumerate(Fs)
        ret1 = rayintersection(ptest1)(V, FV, f)
        ret2 = rayintersection(ptest2)(V, FV, f)
        if typeof(ret1) == Array{Float64,1}
            push!(dep1, (face, ret1))
        end
        if typeof(ret2) == Array{Float64,1}
            push!(dep2, (face, ret2))
        end
    end


    k1, k2 = 0, 0
    for (face, point3d) in dep1
        vs, edges, point2d = planemap(V, copEV, copFE, face)(point3d)
        classify = pointInPolygonClassification(vs, edges)
        inOut = classify(point2d)
        if inOut != "p_out"
            k1 += 1
            push!(intersectedfaces, face)
        end
    end
    if k1 % 2 == 1
        return ptest1, intersectedfaces
    else
        for (face, point3d) in dep2
            vs, edges, point2d = planemap(V, copEV, copFE, face)(point3d)
            classify = pointInPolygonClassification(vs, edges)
            inOut = classify(point2d)
            if inOut != "p_out"
                k2 += 1
                push!(intersectedfaces, face)
            end
        end
        if k2 % 2 == 1
            return ptest2, intersectedfaces
        else
            println("error: while computing internal point of 3-cell")
        end
    end
end

function chainbasis2solids(V, copEV, copFE, copCF)
    CF = [findnz(copCF[k, :])[1] for k = 1:copCF.m]
    FE = [findnz(copFE[k, :])[1] for k = 1:copFE.m]
    EV = [findnz(copEV[k, :])[1] for k = 1:copEV.m]

    FEs = []
    EVs = []
    FVs = []
    for k = 1:copCF.m
        push!(FEs, [collect(Set(GL.Cat([e for e in FE[f]]))) for f in CF[k]])
        push!(EVs, [[EV[e] for e in FE[f]] for f in CF[k]])
        push!(FVs, [collect(Set(GL.Cat([EV[e] for e in FE[f]]))) for f in CF[k]])
    end
    pols = collect(zip(EVs, FVs, FEs))
    W = convert(Lar.Points, V')
    return W, pols, CF
end



function internalpoints(V, copEV, copFE, copCF)

    U, pols, CF = chainbasis2solids(V, copEV, copFE, copCF)

    innerpoints = []
    intersectedfaces = []
    for k = 1:length(pols)
        (EV, FV, FE), Fs = pols[k], CF[k]
        EV = convert(Lar.Cells, collect(Set(GL.Cat(EV))))
        points, facenumber = getinternalpoint(V, EV, FV, Fs, copEV, copFE)
        push!(innerpoints, points)
        push!(intersectedfaces, facenumber)
    end
    return innerpoints, intersectedfaces
end

function bool3d(assembly)

    V, FV, EV = Lar.struct2lar(assembly)
    cop_EV = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
    cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells)
    W = convert(Lar.Points, V')

    V, copEV, copFE, copCF = Lar.space_arrangement(W, cop_EV, cop_FE)
    W = convert(Lar.Points, V')

    innerpoints, intersectedfaces = internalpoints(W, copEV, copFE, copCF[2:end, :])

    listOfModels = Lar.evalStruct(assembly)
    inputfacenumbers = [length(listOfModels[k][2]) for k = 1:length(listOfModels)]
    cumulative = cumsum([0; inputfacenumbers]) .+ 1
    fspans = collect(zip(cumulative[1:end-1], cumulative[2:end] .- 1))
    span(h) = [j for j = 1:length(fspans) if fspans[j][1] <= h <= fspans[j][2]]

    V, FV, EV = Lar.struct2lar(assembly)
    containmenttest = testinternalpoint(V, EV, FV)

    boolmatrix = BitArray(undef, length(innerpoints) + 1, length(fspans) + 1)
    boolmatrix[1, 1] = 1
    for (k, point) in enumerate(innerpoints)
        cells = containmenttest(point)
        rows = [span(h) for h in cells]
        for l in GL.Cat(rows)
            boolmatrix[k+1, l+1] = 1
        end
    end
    return W, (copEV, copFE, copCF), boolmatrix
end

## Esempio versione originale

In [ ]:
using LinearAlgebraicRepresentation, ViewerGL, SparseArrays
Lar = LinearAlgebraicRepresentation;
GL = ViewerGL;
using Base.Threads

n, m, p = 1, 1, 1
V, (VV, EV, FV, CV) = Lar.cuboidGrid([n, m, p], true)
cube = V, FV, EV

assembly = Lar.Struct([ cube,
    Lar.t(.3,.4,.25), Lar.r(pi/5,0,0), Lar.r(0,0,pi/12), cube,
    Lar.t(-.2,.4,-.2), Lar.r(0,pi/5,0), Lar.r(0,pi/12,0), cube,
    Lar.t(-.2,.2,-.1), Lar.r(0,pi/5,0), Lar.r(0,pi/12,0), cube,
])

V, FV = Lar.struct2lar(assembly)

meshes = []
for k = 1:length(FV)
    color = GL.MayaColors[k%12+1] - (rand(Float64, 4) * 0.1)
    push!(meshes, GL.GLGrid(V, [FV[k]], color, 0.9))
end
GL.VIEW(meshes);

W, (copEV, copFE, copCF), boolmatrix = bool3d(assembly)
Matrix(boolmatrix)

A = boolmatrix[:, 2]
B = boolmatrix[:, 3]
C = boolmatrix[:, 4]

AorBorC = A .| B .| C
AandBandC = A .& B .& C
AxorBxorC = A .⊻ B .⊻ C
AminBminC = .&(A, .!B, .!C)

difference = Matrix(copCF)' * Int.(AandBandC)
xor = Matrix(copCF)' * Int.(AxorBxorC)
or = Matrix(copCF)' * Int.(AorBorC)
min = Matrix(copCF)' * Int.(AminBminC)

V, CVs, FVs, EVs = Lar.pols2tria(W, copEV, copFE, copCF, difference) # part of assembly

GL.VIEW(GL.GLExplode(V, FVs, 1.01, 1.01, 1.01, 99, 0.5));
GL.VIEW(GL.GLExplode(V, EVs, 1, 1, 1, 1, 1));

## Performance versione originale

In [ ]:
using BenchmarkTools
@btime bool3d($assembly)

## Esempio del codice sviluppato e ottimizzato
Per mostrare l'esecuzione del codice sviluppato e ottimizzato, è stato creato un package specifico disponibile nel seguente link [Github](https://github.com/paolomazzitti/CPD22-9/tree/studio-esecutivo)

In [7]:
using Pkg
Pkg.add(url="https://github.com/paolomazzitti/CPD22-9#studio-esecutivo")

Pkg.Types.PkgError: expected a `name` entry in project file at `/var/folders/97/mhhx_fj56bn4dsb921y3b4r80000gn/T/jl_jB0OxP/Project.toml`

In [ ]:
using CPD9, ViewerGL, SparseArrays
GL = ViewerGL;
using Base.Threads

T = 3

n, m, p = 1, 1, 1
V, (VV, EV, FV, CV) = CPD9.cuboidGrid([n, m, p], true)
cube = V, FV, EV

assembly = CPD9.Struct([ cube,
    CPD9.t(.3,.4,.25), CPD9.r(pi/5,0,0), CPD9.r(0,0,pi/12), cube,
    CPD9.t(-.2,.4,-.2), CPD9.r(0,pi/5,0), CPD9.r(0,pi/12,0), cube,
    CPD9.t(-.2,.2,-.1), CPD9.r(0,pi/5,0), CPD9.r(0,pi/12,0), cube
])

V, FV = CPD9.struct2lar(assembly)

meshes = []
for k = 1:length(FV)
    color = GL.MayaColors[k%12+1] - (rand(Float64, 4) * 0.1)
    push!(meshes, GL.GLGrid(V, [FV[k]], color, 0.9))
end
GL.VIEW(meshes);

W, (copEV, copFE, copCF), boolmatrix = CPD9.bool3d(assembly)
Matrix(boolmatrix)

A = boolmatrix[:, 2]
B = boolmatrix[:, 3]
C = boolmatrix[:, 4]

AorBorC = A .| B .| C
AandBandC = A .& B .& C
AxorBxorC = A .⊻ B .⊻ C
AminBminC = .&(A, .!B, .!C)

difference = Matrix(copCF)' * Int.(AandBandC)
xor = Matrix(copCF)' * Int.(AxorBxorC)
or = Matrix(copCF)' * Int.(AorBorC)
min = Matrix(copCF)' * Int.(AminBminC)

V, CVs, FVs, EVs = CPD9.pols2tria(W, copEV, copFE, copCF, difference) # part of assembly

GL.VIEW(GL.GLExplode(V, FVs, 1.01, 1.01, 1.01, 99, 0.5));
GL.VIEW(GL.GLExplode(V, EVs, 1, 1, 1, 1, 1));

## Performance del codice sviluppato ed ottimizzato

In [ ]:
using BenchmarkTools
@btime CPD9.bool3d($assembly)